In [5]:
import pandas as pd

# Replace these with your actual Excel file path
excel_file = 'Destatis_Germany Population Historics and Forecasts.xlsx'

# Define sheet names
forecast_sheet = 'Bevölkerung forecasts_ALL VAR'
historical_sheet = 'Bevölkerung historics_per BL'

In [9]:
def inspect_excel_sheet(file_path, sheet_name, label):
    print(f"\n--- Inspecting Sheet: {label} ({sheet_name}) ---")

    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    except Exception as e:
        print(f"❌ Error reading '{sheet_name}' from '{file_path}': {e}")
        return

    print(f"\n✅ Sheet loaded: {sheet_name} — Shape: {df.shape}")
    
    print("\n🔍 First 15 Raw Rows (including metadata, if any):")
    print(df.head(15).to_string(index=False, header=False))

In [10]:
# Inspect both sheets
inspect_excel_sheet(excel_file, forecast_sheet, "Forecast Population Data")


--- Inspecting Sheet: Forecast Population Data (Bevölkerung forecasts_ALL VAR) ---

✅ Sheet loaded: Bevölkerung forecasts_ALL VAR — Shape: (8905, 102)

🔍 First 15 Raw Rows (including metadata, if any):
Vorausberechneter Bevölkerungsstand: Deutschland, Stichtag,\nVarianten der Bevölkerungsvorausberechnung, Geschlecht,\nAltersjahre                                          NaN      NaN          NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN        NaN NaN 

In [11]:
inspect_excel_sheet(excel_file, historical_sheet, "Historical Population Data")


--- Inspecting Sheet: Historical Population Data (Bevölkerung historics_per BL) ---

✅ Sheet loaded: Bevölkerung historics_per BL — Shape: (1034, 18)

🔍 First 15 Raw Rows (including metadata, if any):
Bevölkerung: Bundesländer, Stichtag, Altersjahre               NaN    NaN    NaN         NaN    NaN     NaN    NaN                    NaN           NaN                 NaN             NaN      NaN     NaN            NaN                NaN       NaN                                                                                                      NaN
          Fortschreibung des Bevölkerungsstandes               NaN    NaN    NaN         NaN    NaN     NaN    NaN                    NaN           NaN                 NaN             NaN      NaN     NaN            NaN                NaN       NaN https://www-genesis.destatis.de/genesis/online?sequenz=statistikTabellen&selectionname=12411#abreadcrumb
                      Bevölkerungsstand (Anzahl)               NaN    NaN    NaN         N

In [22]:
import pandas as pd
from googletrans import Translator
import re
import numpy as np

In [26]:
df = pd.read_excel(excel_file, forecast_sheet, header=None)  # Use header=None if there is no header row

# Assume the first column (column 0) contains 'männlich', 'weiblich', 'Insgesamt', or is blank
keep_rows = []
current_mode = None  # Can be 'keep' or 'skip'

for _, row in df.iterrows():
    first_cell = str(row.iloc[2]).strip().lower()

    if first_cell == 'insgesamt':
        current_mode = 'keep'
        keep_rows.append(True)
    elif first_cell in ['männlich', 'weiblich']:
        current_mode = 'skip'
        keep_rows.append(False)
    else:
        # If the first cell is empty, continue current mode
        keep_rows.append(current_mode == 'keep')

# Filter and save the result
filtered_df = df[pd.Series(keep_rows)].reset_index(drop=True)
filtered_df.to_excel("only_insgesamt.xlsx", index=False, header=False)

print("Filtered to keep only 'Insgesamt' blocks.")


Filtered to keep only 'Insgesamt' blocks.


In [27]:
print(filtered_df)

                                                    0    1          2    \
0                                                   NaN  NaN  Insgesamt   
1                                                   NaN  NaN  Insgesamt   
2                                                   NaN  NaN  Insgesamt   
3                                                   NaN  NaN  Insgesamt   
4                                                   NaN  NaN  Insgesamt   
...                                                 ...  ...        ...   
2977                                    danach konstant  NaN        NaN   
2978  W3: Rückgang von 1,5 Mill. in 2022 auf 350.000...  NaN        NaN   
2979                                    danach konstant  NaN        NaN   
2980    W0: Wanderungssaldo Null (keine Außenwanderung)  NaN        NaN   
2981  © Statistisches Bundesamt (Destatis), 2025 | S...  NaN        NaN   

               3      4    5      6    7      8    9    ...    92   93   \
0     unter 1 Jahr    75

In [ ]:
subset = df.iloc[6:, [0, 1]].dropna()  # Drop any rows with NaN in either column

# Step 2: Filter out rows where either column is empty after stripping
subset = subset[(subset[0].astype(str).str.strip() != '') & (subset[1].astype(str).str.strip() != '')]

# Step 3: Drop duplicates to get unique pairs
unique_pairs = subset.drop_duplicates().reset_index(drop=True)

# Print for debug
print("Unique (col0, col1) pairs:")
print(unique_pairs)

# Step 4: Create a new DataFrame from the original one (or a different one)
# For demonstration, let's assume we're copying from the original file

Unique (col0, col1) pairs:
                  0                                                  1
0   BEV-VARIANTE-01       Geburten und LE moderat, WS niedrig (G2L2W1)
1   BEV-VARIANTE-02               Geburten, LE und WS moderat (G2L2W2)
2   BEV-VARIANTE-03          Geburten und LE moderat, WS hoch (G2L2W3)
3   BEV-VARIANTE-04                  Relativ alte Bevölkerung (G1L3W1)
4   BEV-VARIANTE-05                 Relativ junge Bevölkerung (G3L1W3)
5   BEV-VARIANTE-06               Niedrige Geburtenhäufigkeit (G1L2W2)
6   BEV-VARIANTE-07                   Hohe Geburtenhäufigkeit (G3L2W2)
7   BEV-VARIANTE-08      Geringe Steigung der Lebenserwartung (G2L1W2)
8   BEV-VARIANTE-09       Starke Steigung der Lebenserwartung (G2L3W2)
9   BEV-VARIANTE-10                       Bevölkerungsminimum (G1L1W1)
10  BEV-VARIANTE-11                       Bevölkerungsmaximum (G3L3W3)
11  BEV-VARIANTE-12  Geburten niedrig, LE moderat, WS niedrig (G1L2W1)
12  BEV-VARIANTE-13            Lebenserwartung und

In [33]:
new_df = pd.read_excel("only_insgesamt.xlsx", header=None)

In [34]:
# Step 5: Insert the unique pairs only into rows where column 2 == 'Insgesamt'
target_rows = new_df[2].astype(str).str.strip().str.lower() == 'insgesamt'

# Make sure the number of "Insgesamt" rows is at least the number of unique pairs
insgesamt_indices = new_df[target_rows].index.tolist()
num_to_insert = min(len(unique_pairs), len(insgesamt_indices))

# Step 6: Insert the values
for i in range(num_to_insert):
    new_df.at[insgesamt_indices[i], 0] = unique_pairs.iloc[i, 0]
    new_df.at[insgesamt_indices[i], 1] = unique_pairs.iloc[i, 1]

# Save to Excel
new_df.to_excel("updated_with_unique_pairs.xlsx", index=False, header=False)

print("Done! File saved as 'updated_with_unique_pairs.xlsx'.")

C:\Users\Usaid\AppData\Local\Temp\ipykernel_14576\3692631689.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Geburten und LE moderat, WS niedrig (G2L2W1)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_df.at[insgesamt_indices[i], 1] = unique_pairs.iloc[i, 1]


Done! File saved as 'updated_with_unique_pairs.xlsx'.


In [40]:
# Starting from column index 5
start_idx = 5
columns_to_check = new_df.columns[start_idx:]

# Identify columns with even *position* from that point (i.e., idx 0, 2, 4 from the start_idx)
even_positioned_columns = [
    col for i, col in enumerate(columns_to_check)
    if i % 2 == 0  # relative to start_idx
]

# Drop them
new_df_cleaned = new_df.drop(columns=even_positioned_columns)

# Save the result
new_df_cleaned.to_excel("final_cleaned_output.xlsx", index=False, header=False)

print(f"Correctly removed even-positioned columns from index {start_idx} onward.")


Correctly removed even-positioned columns from index 5 onward.


In [42]:
# Load the Excel file (no header row)
df_v3 = pd.read_excel("final_cleaned_output.xlsx", header=None)

# Define how many columns there are
num_columns = df_v3.shape[1]

# Define where to start adding years (5th column = index 4)
start_col = 4
start_year = 2022

# Construct the top row: empty strings before start_col, then increasing years
top_row = [''] * start_col + [str(start_year + i) for i in range(num_columns - start_col)]

# Create a DataFrame from the row
top_row_df = pd.DataFrame([top_row], columns=df_v3.columns)

# Concatenate the row and original data
df_with_years = pd.concat([top_row_df, df_v3], ignore_index=True)

# Save the result
df_with_years.to_excel("with_year_row.xlsx", index=False, header=False)

print("✅ Added a row of years at the top. Saved as 'with_year_row.xlsx'.")

✅ Added a row of years at the top. Saved as 'with_year_row.xlsx'.


In [45]:
# Load your Excel file (with year header at the top)
df_age = pd.read_excel("with_year_row.xlsx", header=None)

# Separate the year header row
year_header = df_age.iloc[[0]]
data = df_age.iloc[1:].copy().reset_index(drop=True)

age_col = 3        # 4th column (0-based)
start_data_col = 4 # from 5th column onward

# Labels
block_start_age = "unter 1 Jahr"
block_end_age = "100 Jahre und mehr"

start_age_90 = "90-Jährige"
end_age_100 = "100 Jahre und mehr"
insert_after_age = "89-Jährige"

# Find block boundaries (start and end indices)
block_starts = data.index[data[age_col] == block_start_age].tolist()
block_ends = data.index[data[age_col] == block_end_age].tolist()

assert len(block_starts) == len(block_ends), "Mismatch in block starts and ends"

result_blocks = []

for start_idx, end_idx in zip(block_starts, block_ends):
    block = data.loc[start_idx:end_idx].copy().reset_index(drop=True)
    
    # Find positions of start_age_90 and end_age_100 inside block
    try:
        idx_start_90 = block[block[age_col] == start_age_90].index[0]
    except IndexError:
        # No 90 age found in this block, just keep block as is
        result_blocks.append(block)
        continue

    try:
        idx_end_100 = block[block[age_col] == end_age_100].index[0]
    except IndexError:
        # No 100 age found, use last row as end
        idx_end_100 = block.index[-1]

    # Rows to sum = from idx_start_90 to idx_end_100 inclusive
    rows_to_sum = block.loc[idx_start_90:idx_end_100, start_data_col:]
    summed_vals = rows_to_sum.apply(pd.to_numeric, errors='coerce').sum()

    # Create new row
    new_row = [''] * age_col + ["90 Jahre und mehr"] + summed_vals.tolist()

    # Drop the old 90+ rows
    block_cleaned = block.drop(index=range(idx_start_90, idx_end_100+1)).reset_index(drop=True)

    # Find insert position (after '89-Jährige')
    try:
        insert_pos = block_cleaned[block_cleaned[age_col] == insert_after_age].index[0] + 1
    except IndexError:
        # If no '89-Jährige' found, append at end
        insert_pos = len(block_cleaned)

    # Insert new row
    block_with_new_row = pd.concat([
        block_cleaned.iloc[:insert_pos],
        pd.DataFrame([new_row], columns=block.columns),
        block_cleaned.iloc[insert_pos:]
    ]).reset_index(drop=True)

    result_blocks.append(block_with_new_row)

# Combine all blocks back
final_data = pd.concat(result_blocks, ignore_index=True)

# Add year header row back
final_df = pd.concat([year_header, final_data], ignore_index=True)

# Save result
final_df.to_excel("final_with_correct_summed_90plus_per_block.xlsx", index=False, header=False)

print("✅ Processed blocks: summed 90+ ages, inserted new row, removed originals correctly.")

✅ Processed blocks: summed 90+ ages, inserted new row, removed originals correctly.


In [47]:
df_final = pd.read_excel("final_with_correct_summed_90plus_per_block.xlsx", header=None)

# Extract header
year_header = df_final.iloc[[0]]
data = df_final.iloc[1:].copy().reset_index(drop=True)

# Config
age_col = 3        # Column D (0-based)
data_start_col = 4 # Column E onwards

# Age markers
start_age = "unter 1 Jahr"
end_age = "90 Jahre und mehr"
cutoff_age = "90 Jahre und mehr"

# Find block boundaries
block_starts = data.index[data[age_col] == start_age].tolist()
block_ends = data.index[data[age_col] == end_age].tolist()

assert len(block_starts) == len(block_ends), "Mismatch in age block boundaries!"

new_blocks = []

for start_idx, end_idx in zip(block_starts, block_ends):
    block = data.loc[start_idx:end_idx].copy().reset_index(drop=True)

    try:
        idx_cutoff = block[block[age_col] == cutoff_age].index[0]
    except IndexError:
        print(f"⚠️ Skipping block without cutoff age: {cutoff_age}")
        new_blocks.append(block)
        continue

    # Sum from "unter 1 Jahr" to "90 Jahre und mehr" inclusive
    rows_to_sum = block.loc[0:idx_cutoff, data_start_col:]
    summed_values = rows_to_sum.apply(pd.to_numeric, errors='coerce').sum()

    # Build new row
    new_row = [''] * age_col + ["Insgesamt"] + summed_values.tolist()

    # Insert after "90 Jahre und mehr"
    insert_pos = idx_cutoff + 1
    block_with_sum = pd.concat([
        block.iloc[:insert_pos],
        pd.DataFrame([new_row], columns=block.columns),
        block.iloc[insert_pos:]
    ]).reset_index(drop=True)

    new_blocks.append(block_with_sum)

# Combine blocks back
final_data = pd.concat(new_blocks, ignore_index=True)

# Add back the year row
final_result = pd.concat([year_header, final_data], ignore_index=True)

# Save to Excel
final_result.to_excel("final_with_insgesamt_per_block.xlsx", index=False, header=False)

print("✅ Added 'Insgesamt' row after each '90 Jahre und mehr' row per block.")

✅ Added 'Insgesamt' row after each '90 Jahre und mehr' row per block.
